# Array-Oriented Programming with NumPy - Part 2


1. `array`  Operators

2. Broadcasting 

3. Universal Functions (Vectorization)

4. Type casting and File I/O

In the first step, we need to install `NumPy` as follows:

In [3]:
package_name = "numpy"

try:
    __import__(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} not found. Installing...")
    %pip install {package_name}

numpy is already installed.


The official `NumPy` documentation recommends importing the `numpy` module as `np` so that we can access its methods with `np.`:

In [4]:
import numpy as np

## `array`  Operators

### The slowness of loops

The speed of computations on `NumPy` `arrays` can range from very fast to very slow. To optimize performance, the recommended approach is to use ***vectorized operations***, which are typically implemented through `NumPy`'s universal functions (`ufuncs`). 

In scenarios involving numerous small operations executed repeatedly, the inherent latency of Python often becomes noticeable. This is particularly the case when looping over arrays to perform operations on each element.

For instance, suppose we have an array of values and need to compute the reciprocal of each. A straightforward approach to this could involve:

In [1]:
def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output

In [5]:
values = np.random.randint(1, 10, 5)
print(values)
compute_reciprocals(values)

[2 2 2 1 9]


array([0.5       , 0.5       , 0.5       , 1.        , 0.11111111])

But if we measure the execution time of this code for a large input, we see that this operation is very slow:

In [6]:
big_array = np.random.randint(1, 10, 1_000_000)

In [7]:
%%timeit 
compute_reciprocals(big_array)

1.15 s ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


> Interestingly, the bottleneck in this situation isn't the operations themselves, but rather the type checking and function dispatches that `Python` needs to execute during each iteration of the loop. Whenever the reciprocal is calculated, `Python` initially verifies the type of the object and performs a **dynamic lookup to determine the correct function to employ for that type**. If we were using compiled code, this kind of specification would be predetermined before the code execution, resulting in much more efficient computations.

In `NumPy`, ***vectorization*** is the process of performing operations on entire `arrays` of data, as opposed to individual elements. This is accomplished by applying an operation to the entire `array`, instead of looping through each element of the `array` one at a time.

In [8]:
print(values)
1.0 / values # The vectorized version of the above code

[2 2 2 1 9]


array([0.5       , 0.5       , 0.5       , 1.        , 0.11111111])

The above syntax is the vectorized version of the original code and works due to the ***broadcasting***.

Looking at the execution time for our big `array`, we see that it completes orders of magnitude faster than the `Python` loop:

In [9]:
%%timeit 
(1.0 / big_array)

2 ms ± 48.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The execution time is much faster since the vectorization operation is done via `ufuncs`, which is a compiled routine.

In the next section, we will introduce each concept in detail, including broadcasting, `ufuncs` and vectorization.

#### Element-wise arithmetic

`NumPy` offers numerous operators that allow us to create simple expressions that carry out operations on whole arrays and returns another `array`. Firstly, let's perform **element-wise arithmetic with arrays and a scalar value** by employing arithmetic operators and <u>augmented assignments</u>.

Element-wise operations are applied to each element, so the snippet below doubles every element and cubes every element. Each operation returns a new array containing the result:

In [10]:
numbers = np.arange(1, 7) # array([1, 2, 3, 4, 5, 6])
numbers * 2

array([ 2,  4,  6,  8, 10, 12])

In [11]:
numbers ** 3

array([  1,   8,  27,  64, 125, 216], dtype=int32)

Augmented assignments modify every element in the left operand in place!

In [12]:
numbers += 10
numbers

array([11, 12, 13, 14, 15, 16])

### Example 1: Given the function $f(x) = x^2 + 2x + 1$, estimate the integral of $f(x)$ from $x = 0$ to $x = 2$ using the Riemann sum.

A Riemann sum is a specific kind of approximation of an integral by a finite sum. It is computed as follows:

Given a function $f(x)$, and a partition of the interval $[a, b]$ into $N$ subintervals, denoted by:

$$
x_0 = a < x_1 < \cdots < x_{N-1} < x_N = b
$$

A Riemann sum of this function is constructed as:

$$
\sum_{i=1}^N f(x_i^{'}) (x_i - x_{i-1}) \ \ , \ \text{where} \ x_i^{'} \in [x_{i-1},x_i]
$$

Here, $x_i^{'}$ is an arbitrary point within each subinterval $[x_{i-1},x_i]$.

Riemann sums hold significant importance as they allow us to easily approximate a definite integral, represented as:

$$
\int_a^b f(x) \, dx \approx \sum_{i=1}^N f(x_i^{'}) (x_i - x_{i-1}) \ \ , \ \text{where} \ x_i^{'} \in [x_{i-1},x_i]
$$

In [ ]:
# 1. Define the function

# 2. Generate x values

# 3. Compute y values

# 4. Estimate the integral using the Riemann sum

# The exact results
exact_integral = (2**3)/3 + (2**2) + 2
print(f"Exact integral: {exact_integral}")

### Broadcasting 

Typically, arithmetic operations necessitate two `arrays` of identical size and shape as operands. When one operand is a scalar, `NumPy` carries out the element-wise calculations as though the scalar were an array of the same shape as the other operand, but with the scalar value present in all its elements. 

This is referred to as ***broadcasting***. For instance, `numbers * 2` is equivalent to `numbers * [2, 2, 2, 2, 2, 2]`.

Broadcasting can also be applied between `arrays` of different sizes and shapes, enabling concise and powerful manipulations. We will present more examples of broadcasting later in this chapter when we introduce `NumPy`'s universal functions.

#### Arithmetic Operations Between `arrays`

Arithmetic operations and augmented assignments can be performed between arrays of the same shape. Let's multiply the one-dimensional arrays `numbers` and `numbers2`, each containing six elements:

In [ ]:
import numpy as np
numbers = np.array([11, 12, 13, 14, 15, 16])
numbers2 = np.linspace(1.1, 6.6, 6) 
numbers * numbers2 # array([11, 12, 13, 14, 15, 16]) * array([ 1.1,  2.2,  3.3,  4.4,  5.5, 6.6])

The outcome is a new `array` created by multiplying the elements of each operand element-wise — `11 * 1.1, 12 * 2.2, 13 * 3.3`, and so on. Arithmetic operations between `arrays` of integers and floating-point numbers result in an `array` of floating-point numbers due to <u> type conversion </u>. 

Let's see another example:

In [ ]:
c = np.ones((3, 3))
c * c 

Note that the above operation is not matrix multiplication. To perform matrix multiplication use the `dot()` method!

In [ ]:
c.dot(c)

The above operation is the same as using the `@` operator:

In [ ]:
c @ c

We can apply broadcasting to `arrays` with different shape. For instance, consider adding a one-dimensional `array` to a two-dimensional `array` and observe the resulting output:

In [ ]:
a = np.array([0, 1, 2]) 
M = np.ones((3, 3))
print(a.shape, M.shape) # Note a is not (3,1) or (1,3)
M + a

Here, the one-dimensional array `a` is stretched, or broadcasted, across the second dimension in order to match the shape of `M`.

#### Rules of Broadcasting

In `NumPy`, broadcasting adheres to a strict set of regulations that govern how two `arrays` interact with one another. These rules are as follows:

1. When the number of dimensions between two `arrays` differs, the `array` with fewer dimensions is padded with ones on its leading (left) side to match the number of dimensions of the other `array`.
2. If the shape of the two `arrays` doesn't match in any dimension, the `array` with a shape of 1 in that dimension is expanded to match the shape of the other `array`.
3. If the sizes of the `arrays` conflict in any dimension and neither is equal to 1, an error is raised.

Now let's take a look at an example where both `arrays` need to be broadcast:

In [ ]:
a = np.arange(0, 40, 10).reshape(4,1)
b = np.arange(3)
print(a.shape, b.shape)
a, b

In [ ]:
a + b

1. To begin, we need to determine the shapes of the two `arrays`: `a.shape` is `(4,1)` and `b.shape` is `(3,)`. According to Rule 1, we have to add ones to the shape of `b` such that its dimensions match those of `a`. Thus, `b.shape` becomes `(1,3)`.

2. Next, Rule 2 states that we need to expand each of the 1s in `b.shape` to match the corresponding size of the other `array`. Consequently, `a.shape` becomes `(4,3)`, and `b.shape` becomes `(4,3)` since 1 was replicated three times to match the size of `a`.

3. Since the shapes of the two `arrays` now match, they are compatible. 

This entire process can be depicted visually as follows:

<center><img src="https://drive.google.com/uc?id=1zKGHApvzo2NvyuB2vLfu6en9WI0_QCUj" width="70%" height="70%"></center>

Next, let's look at an example in which the two `arrays` are incompatible!

In [ ]:
M = np.ones((3, 2))
a = np.arange(3)

M.shape, a.shape

1. First, we need to determine the shapes of the two `arrays`: `M.shape` is `(3,2)`, and `a.shape` is `(3,)`. As per Rule 1, we must pad ones to the shape of `a` such that its number of dimensions matches that of `M`. Consequently, `a.shape` becomes `(1, 3)`, while `M.shape` remains the same.

2. Next, Rule 2 requires that we stretch the first dimension of `a` to match that of `M`. Therefore, `a.shape` becomes `(3,3)`, while `M.shape` stays the same.

3. However, Rule 3 comes into play, here since the final shapes of the two `arrays` do not match. As a result, these two `arrays` are incompatible. 

This incompatibility is evident when we attempt to perform this operation.

In [ ]:
M + a

### Exercise 1: Suppose we are dealing with a spreadsheet that records the grade of students. The grade contains the homework, midterm and finals as follows:

<div align="center">

| Name | HW1 | HW2 | HW3 | HW4 | Midterm | Final |
| --- | --- | --- | --- | --- | --- | --- |
| Alice | 90 | 80 | 70 | 100 | 90 | 95 |
| Bob | 80 | 90 | 100 | 70 | 85 | 80 |
| Charlie | 70 | 100 | 90 | 80 | 95 | 90 |
| David | 60 | 70 | 80 | 90 | 85 | 100 |
| Eve | 50 | 60 | 70 | 80 | 75 | 90 |

</div>

We would like to calculate the semester score of each student by the following rules:

1. The weight of each score is 0.2 (the summation of four homework accounts for 20% of the total scores and each homework has the same weight), 0.4 and 0.4 for HW, Midterm and Final, respectively. 

2. We adjust each student's score so that the top performer in the class gets a score of 100 by adding the same constant score to each student's score.

We use a 2D array to model the grades so that each row corresponds to a student's score. Use the following template to complete the task:

```python
grades = np.array([[90, 80, 70, 100, 90, 95],
                   [80, 90, 100, 70, 85, 80],
                   [70, 100, 90, 80, 95, 90],
                   [60, 70, 80, 90, 85, 100],
                   [50, 60, 70, 80, 75, 90]])

weights = np.array([])
scores
```

In [ ]:
# input data
grades = np.array([[90, 80, 70, 100, 90, 95],
                   [80, 90, 100, 70, 85, 80],
                   [70, 100, 90, 80, 95, 90],
                   [60, 70, 80, 90, 85, 100],
                   [50, 60, 70, 80, 75, 90]])

# weights of HW, Midterm, Final
weights = np.array([])

# calculate weighted score
scores

### Universal Functions (Vectorization)

Now we will delve into how `NumPy` perform element-wise operations on `arrays` without using the `for` loop: `NumPy` provides most operators/functions as standalone ***universal functions*** (`ufuncs`) that perform various operations element-wise, meaning that they apply the same operation to each element in an `array`. 

These functions operate on one or two `array`-like arguments and are utilized to perform tasks. Some of these functions are automatically invoked when operators like `+` and `*` are used with `arrays`. Each `ufunc` generates a new `array` that contains the results of the operation.

`NumPy` offers a practical interface that directly access these <u>statically typed</u> and <u>compiled routines</u>. These operations are called ***vectorized operations***. Vectorization is achieved using `array` operations, such as addition, subtraction, multiplication, and division. In addition, it can also be achieved by using other `ufunc`. 

These vectorized methods are intended to move the loop to the compiled layer that underpins `NumPy`, leading to considerably quicker execution.

We can view the complete list, their descriptions and more information about universal functions at [https://numpy.org/doc/stable/reference/ufuncs.html](https://numpy.org/doc/stable/reference/ufuncs.html)

> See [here](https://www.labri.fr/perso/nrougier/from-python-to-numpy/#problem-vectorization) for more vectorization examples in different thinking levels.

#### Exploring `NumPy`’s `Ufuncs`

Let's add two `arrays` with the same shape, using the `add()` universal function:

In [ ]:
import numpy as np
numbers = np.array([11, 12, 13, 14, 15, 16])
numbers2 = np.arange(10, 70, 10)  # array([10, 20, 30, 40, 50, 60])
np.add(numbers, numbers2)        # equivalent to numbers + numbers2

#### Broadcasting with Universal Functions

Let's use the `multiply()` universal function to multiply every element of `numbers2` by the scalar value 5:

In [ ]:
np.multiply(numbers2, 5) # equivalent to numbers2 * 5

Let's reshape `numbers2` into a 2-by-3 array, then multiply its values by a one-dimensional `array` of three elements:

In [ ]:
numbers3 = numbers2.reshape(2, 3)
numbers4 = np.array([2, 4, 6])
numbers3, numbers4

In [ ]:
np.multiply(numbers3, numbers4) # Equivalent to numbers3 * numbers4

In this case, `numbers4` has the same length as each row of `numbers3`, allowing `NumPy` to apply the multiplication operation by treating `numbers4` as an `array` with the following values:

```python
array([[2, 4, 6],
       [2, 4, 6]])
```

If a universal function receives two `arrays` with different shapes that do not support broadcasting, a `ValueError` is raised.

There are other special mathematical `ufunc`. Let's create an array and the values using the `sin()` universal function:

In [ ]:
numbers = np.array([1, 4, 9, 16, 25, 36])
np.sin(numbers)

Vectorization and `ufunc` functions are closely associated with broadcasting in `NumPy`. By combining vectorization, `ufunc` functions, and broadcasting, we can effectively execute complex arithmetic operations on `NumPy` `arrays`. 

However, it's important to mention that vectorization can be achieved through methods other than just using ufuncs.

#### Create our own vectorizing functions

The vectorized operation are often more concise, and it is thus advisable to avoid element-wise looping over vectors and matrices and instead employ vectorized algorithms. 

The first step in converting a scalar algorithm to a vectorized algorithm involves verifying that the functions we create can function with vector inputs:

In [ ]:
def Theta(x, th):
    """
    Scalar implemenation of a variant of Heaviside step function.
    """
    if x >= th:
        return 1
    else:
        return 0

We can achieve this using `np.vectorize()` function:

In [ ]:
Theta_vec = np.vectorize(Theta)
Theta_vec(np.array([-3,-2,-1,0,1,2,3]), 1)

> Don’t assume `np.vectorize()` is faster. It is mainly for convenient and concise purposes as described [here](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html).

### Exrercise 2: Compare the performance between `for` loop and `NumPy` vectorization in calculating the Wallis formula: $2 \times \prod_{i=1}^{500}(\frac{2i}{2i-1}\times\frac{2i}{2i+1})$. Be sure to check the results from the two approaches are the same and close enough to the true value of $\pi$. Finally, report the speedup factor of the `NumPy` vectorization. (10%)

Hint: Use `%%timeit` to measure the performance of the code. In addition, look up the official documentation or use copilot to find the `NumPy` function to calculate the product of an array using vectorization.

In [ ]:
# Your code here

In [ ]:
# Your code here

## Type casting and File I/O

### Type casting

Due to the nature of static typing, the type of a `NumPy` `array` does not change once created. However, we can explicitly convert an `array` of one type to another using the `astype()` function. This operation always generates a new `array` with a new type.

In [ ]:
import numpy as np
M = np.random.rand(5,5)*5
M, M.dtype

In [ ]:
M2 = M.astype(np.int64)
M2

See [https://scipy-lectures.org/intro/numpy/elaborate_arrays.html](https://scipy-lectures.org/intro/numpy/elaborate_arrays.html) for more details.

### File I/O

Let's consider that we have a two-dimensional `array` representing student grades. Each row in the array corresponds to a unique student, and each column represents a different subject:

In [ ]:
grades = np.array([[90, 80, 70, 100, 90, 95],
                   [80, 90, 100, 70, 85, 80],
                   [70, 100, 90, 80, 95, 90],
                   [60, 70, 80, 90, 85, 100],
                   [50, 60, 70, 80, 75, 90]])

We can store it into a text file using `np.savetxt()`:

In [ ]:
np.savetxt('grades.txt', grades)

On the other hand, we can read the contents from a text file using `np.loadtxt()`:

In [ ]:
grades2  = np.loadtxt('grades.txt')
print(grades2)

`NumPy` has its own binary format, not portable but with efficient I/O. This is useful when storing and reading back large `array` data. Use the functions `np.save()` and `np.load()`.

In [ ]:
M = np.random.randint(1,6, size=(1_000, 1_000))
print(M)
np.save("random-matrix.npy", M)

In [ ]:
M2 = np.load("random-matrix.npy")
M2

In summary:

To make the code faster using `NumPy`  

- Use views instead of copies whenever possible
- Broadcasting: Use broadcasting to do operations on `arrays`
- Vectorizing `for` loops: Find tricks to avoid `for` loops using `NumPy` `arrays`.
- In place operations: `a *= 3` instead of `a = 3*a`

The comparisons between `list` and `array` are summarized as follows:

**Python** objects:

- `Python` `lists` are very general. They can contain any kind of object and are dynamically typed 
- However, they do not support mathematical functions such as matrix multiplications. Implementing such functions for `Python` `lists` would not be very efficient because of the dynamic typing

**NumPy** provides:

- `Numpy` `arrays` are **statically typed** and **have the same data type**. The type of the elements is determined when the `array` is created
- Because of this static typing, `NumPy` can utilize fast implementation of mathematical functions using a compiled language (NumPy uses C and Fortran). This contributes to their computational and memory efficiency.
- For scientific computing tasks where efficiency and mathematical operations are key, it is generally recommended to use `NumPy` arrays to model and manipulate the data.

## References

1. [https://scipy-lectures.org/intro/numpy/index.html](https://scipy-lectures.org/intro/numpy/index.html)

2. [https://scipy-lectures.org/advanced/advanced_numpy/index.html](https://scipy-lectures.org/advanced/advanced_numpy/index.html)

## Key terms

- **Vectorization**: This is a technique in computing which involves performing operations on entire arrays instead of individual elements, thus significantly improving computational efficiency. It's commonly used in numerical computing environments like NumPy.
- **Augmented assignment**: In programming, augmented assignment is a kind of shorthand that combines an arithmetic operation with an assignment. For example, instead of writing x = x + 1, you can write x += 1, which does the same thing.
- **Broadcasting**: This is a technique used in programming, particularly with numerical computing libraries such as NumPy, that allows operations to be performed between arrays of different shapes and sizes. It does this by 'broadcasting' the smaller array across the larger one so that they have compatible shapes.
- **Type conversion**: Type conversion, or type casting, is changing a value from one data type to another. It can be automatic (implicit), like when an integer is automatically converted to a float for a calculation, or manual (explicit), as when a programmer uses a function like int() to convert a string to an integer in Python.
- **Universal functions**: ufuncs in NumPy operate element-wise on arrays, enabling faster computations than traditional loops in Python. The reason they're faster is because they're implemented in C, a compiled and more efficient language, and the Python layer simply provides an interface to the compiled code. Hence, ufuncs use the concept of compiled routines to achieve better performance.
- **Statically typed**: A language is statically typed if the type is checked before the code is executed. This means that you need to declare the type of a variable when it's created, and it can't be changed to another type later. Examples of statically typed languages are C, C++, and Java.
- **Compiled routines**: In programming, routines (or functions) are compiled into machine code before being run in a compiled language. This typically leads to more efficient execution than interpreted languages, which interpret the code line by line as it is run.
- **Type casting**: This is essentially another term for type conversion. It's the process of converting one data type to another. The term is often used in statically typed languages or library like NumPy where the type needs to be explicitly stated.
- **Binary file**: A binary file is a file that contains binary data as opposed to text data. This could be anything from an image or audio file to an executable program. Binary files are read differently than text files and often need specific software to interpret them. On the other hand, binary file often offers efficiency benefits in terms of space and speed as it is the format that resembles the format used in the underlying storage system.